## 第七章：函数

使用 `def` 语句定义函数是所有程序的基础。

本章的目标是讲解一些更加高级和不常见的函数定义与使用模式。

涉及到的内容包括默认参数、任意数量参数、强制关键字参数、注解和闭包。

另外，一些高级的控制流和利用回调函数传递数据的技术在这里也会讲解到。

### 7.1 可接受任意数量参数的函数

* 问题

构造一个可接受任意数量参数的函数。

* 解决方案

为了能让一个函数接受任意数量的**位置参数**，可以使用一个 `*` 参数。例如：

In [1]:
def avg(first, *rest):
    return (first + sum(rest)) / (1 + len(rest))

In [2]:
# Sample use
avg(1, 2)

1.5

In [3]:
avg(1, 2, 3, 4)

2.5

在这个例子中，`rest` 是由所有其他位置参数组成的元组。然后我们在代码中把它当成了一个序列来进行后续的计算。

为了接受任意数量的**关键字参数**，使用一个以`**`开头的参数。比如

In [4]:
import html


def make_element(name, value, **attrs):
    keyvals = [' %s="%s"' % item for item in attrs.items()]
    attr_str = ''.join(keyvals)
    element =  '<{name}{attrs}>{value}</{name}>'.format(
                name=name,
                attrs=attr_str,
                value=html.escape(value))
    return element

# Example
# Creates '<item size="large" quantity="6">Albatross</item>'
make_element('item', 'Albatross', size='large', quantity=6)

# Creates '<p>&lt;spam&gt;</p>'
make_element('p', '<spam>')

'<p>&lt;spam&gt;</p>'

`attrs` 是一个包含所有被传入进来的关键字参数的字典

如果还希望能同时接受任意数量的位置参数和关键字参数，可以同时使用 `*` 和 `**`。

In [5]:
def anyargs(*args, **kwargs):
    print(args)  # A tuple
    print(kwargs)  # A dict

使用这个函数时，所有位置参数会被放到args元组中，所有关键字参数会被放到字典kwargs中。

In [6]:
anyargs(1, 2, 3, 4, 5, large=3, low=5)

(1, 2, 3, 4, 5)
{'large': 3, 'low': 5}


* 讨论

一个`*` 参数只能出现在函数定义中**最后一个位置参数后面**，而 `**` 参数只能是**最后一个参数**。

有一点要注意的是，在`*` 参数后面仍然可以定义其他参数。
```python
def a(x, *args, y):
    pass

def b(x, *args, y, **kwargs):
    pass
```
这种参数就是我们所说的强制关键字参数，在后面7.2小节还会详细讲解到。

### 7.2 只接受关键字参数的函数

* 问题

希望函数的某些参数强制使用关键字参数传递

* 解决方案

将强制关键字参数放到某个`*` 参数或者单个`*` 后面就能达到这种效果。比如：

In [7]:
def recv(maxsize, *, block):
    """Receives a message"""
    print(maxsize)
    print(block)

recv(1024, True)  # TypeError

TypeError: recv() takes 1 positional argument but 2 were given

In [8]:
recv(1024, block=True)  # OK

1024
True


利用这种特性，我们还能在接受任意多个位置参数的函数中指定关键字参数。比如：

In [9]:
def minimum(*values, clip=None):
    m = min(values)
    if clip is not None:
        m = clip if clip > m else m
    return m

In [10]:
minimum(1, 5, 2, -5, 10)  # Returns -5

-5

In [11]:
minimum(1, 5, 2, -5, 10, clip=0)  # Returns 0

0

* 讨论

很多情况下，使用强制关键字参数会比使用位置参数表意更加清晰，程序也更加具有可读性。 例如，考虑下如下一个函数调用：
```python
msg = recv(1024, False)
```

如果调用者对 `recv` 函数并不是很熟悉，那他肯定不明白那个`False` 参数到底来干嘛用的。但是，如果代码变成下面这样子的话就清楚多了：
```python
msg = recv(1024, block=False)
```


另外，使用强制关键字参数也会比使用`**kwargs`参数更好，因为在使用函数help的时候输出也会更容易理解：

In [12]:
help(recv)

Help on function recv in module __main__:

recv(maxsize, *, block)
    Receives a message



强制关键字参数在一些更高级场合同样也很有用。

例如，它们可以被用来在使用 `*args` 和 `**kwargs` 参数作为输入的函数中插入参数，9.11小节有一个这样的例子。

### 7.3 给函数参数增加元信息

* 问题

你写好了一个函数，然后想为这个函数的参数增加一些额外的信息，这样的话其他使用者就能清楚的知道这个函数应该怎么使用。

* 解决方案

使用函数参数注解是一个很好的办法，它能提示程序员应该怎样正确使用这个函数。 例如，下面有一个被注解了的函数：

In [13]:
def add(x: int, y: int) -> int:
    return x + y

python解释器不会对这些注解添加任何的语义。它们不会被类型检查，运行时跟没有加注解之前的效果也没有任何差距。

然而，对于那些阅读源码的人来讲就很有帮助啦。第三方工具和框架可能会对这些注解添加语义。同时它们也会出现在文档中。

In [14]:
help(add)

Help on function add in module __main__:

add(x:int, y:int) -> int



尽管你可以使用任意类型的对象给函数添加注解(例如数字，字符串，对象实例等等)，不过通常来讲使用类或者字符串会比较好点。

* 讨论

函数注解只存储在函数的 `__annotations__` 属性中。例如：

In [15]:
add.__annotations__

{'x': int, 'y': int, 'return': int}

尽管注解的使用方法可能有很多种，但是它们的主要用途还是文档。

因为python并没有类型声明，通常来讲仅仅通过阅读源码很难知道应该传递什么样的参数给这个函数。

这时候使用注解就能给程序员更多的提示，让他们可以正确的使用函数。

参考9.20小节的一个更加高级的例子，演示了如何利用注解来实现多分派(比如重载函数)。

### 7.4 返回多个值的函数

* 问题

希望构造一个可以返回多个值的函数

* 解决方案

为了能返回多个值，函数直接return一个元组就行了。例如：

In [16]:
def myfunc():
    return 1, 2, 3

In [17]:
a, b, c = myfunc()

In [18]:
a

1

In [19]:
b

2

In [20]:
c

3

* 讨论

尽管 `myfun()` 看上去返回了多个值，实际上是**先创建了一个元组然后返回的**。

这个语法看上去比较奇怪，实际上我们使用的是逗号来生成一个元组，而不是用括号。比如下面的：

In [21]:
a = (1, 2)  # 括号生成元组
a

(1, 2)

In [22]:
b = 1, 2  # 不带括号
b

(1, 2)

当我们调用返回一个元组的函数的时候，通常我们会将结果赋值给多个变量，就像上面的那样。

其实这就是1.1小节中我们所说的元组解包。返回结果也可以赋值给单个变量，这时候这个变量值就是函数返回的那个元组本身了

In [23]:
x = myfunc()
x

(1, 2, 3)

In [24]:
type(x)

tuple

### 7.5 定义有默认参数的函数

* 问题

定义一个函数或者方法，它的一个或多个参数是可选的并且有一个默认值。

* 解决方案

定义一个有可选参数的函数是非常简单的，直接在函数定义中给参数指定一个默认值，并放到参数列表最后就行了。

In [25]:
def spam(a, b=42):
    print(a, b)

In [26]:
spam(1)

1 42


In [27]:
spam(1, 2)

1 2


如果默认参数是一个可修改的容器比如一个列表、集合或者字典，可以使用None作为默认值，就像下面这样

In [28]:
# Using a list as a default value
def spam(a, b=None):
    if b is None:
        b = []

如果你并不想提供一个默认值，而是想仅仅测试下某个默认参数是不是有传递进来，可以像下面这样写

In [29]:
_no_value = object()

def spam(a, b=_no_value):
    if b is _no_value:
        print('No b value supplied')

In [30]:
spam(1)

No b value supplied


In [31]:
spam(1, 2)  # b=2

In [32]:
spam(1, None)  # b=None

传递一个 `None值` 和 `不传值` 两种情况是有差别的。

* 讨论

定义带默认值参数的函数是很简单的，但还有一些内容需要深入了解

1. 首先，默认参数的值仅仅在函数定义的时候赋值一次。

In [33]:
x = 42

def spam(a, b=x):
    print(a, b)

In [34]:
spam(1)

1 42


In [35]:
x = 23
spam(1)  # b=42

1 42


注意到当我们改变x的值的时候对默认参数值并没有影响，**这是因为在函数定义的时候就已经确定了它的默认值了。**

2. 其次，默认参数的值应该是不可变的对象，比如`None`、`True`、`False`、数字或字符串。

In [36]:
def spam(a, b=[]): # NO!!!
    pass

如果你这么做了，当默认值在其他地方被修改后你将会遇到各种麻烦。这些修改会影响到下次调用这个函数时的默认值。

In [37]:
def spam(a, b=[]):
    print(b)
    return b

In [38]:
x = spam(1)

[]


In [39]:
x.append(99)
x.append('Yow!')
x

[99, 'Yow!']

In [40]:
spam(1)  # Modified list gets returned!

[99, 'Yow!']


[99, 'Yow!']

为了避免这种情况的发生，最好是将默认值设为None，然后在函数里面检查它。

在测试`None`值时使用 `is` 操作符是很重要的，也是这种方案的关键点。有时候大家会犯下下面这样的错误：

In [41]:
def spam(a, b=None):
    if not b:  # NO! use 'b is None' instead
        b = []
    print(b)

这么写的问题在于尽管 `None` 值确实是被当成 `False`，但是还有其他的对象(比如长度为0的字符串、列表、元组、字典等)都会被当做False。

因此，上面的代码会误将一些其他输入也当成是没有输入。比如：

In [42]:
spam(1)  # ok

[]


In [43]:
x = []
spam(1, x)

[]


In [44]:
spam(1, 0)

[]


In [45]:
spam(1, '')

[]


一个函数需要测试某个可选参数是否被使用者传递进来。

这时候需要小心的是：你不能用某个默认值比如None、0或者False值来测试用户提供的值（因为这些值都是合法的值，是可能被用户传递进来的)。

为了解决这个问题，你可以创建一个独一无二的私有对象实例，就像上面的 `_no_value` 变量那样。

在函数里面，你可以通过检查被传递参数值跟这个实例是否一样来判断。

这里的思路是用户不可能去传递这个 `_no_value` 实例作为输入。

因此，这里通过检查这个值就能确定某个参数是否被传递进来了。

这里对 `object()` 的使用看上去有点不太常见。`object` 是python中所有类的基类。

你可以创建 `object` 类的实例，但是这些实例没什么实际用处，因为它并没有任何有用的方法，

也没有任何实例数据(因为它没有任何的实例字典，你甚至都不能设置任何属性值)。

你唯一能做的就是测试同一性。这个刚好符合我的要求，因为我在函数中就只是需要一个同一性的测试而已。

### 7.6 定义匿名或内联函数

* 问题

为 `sort()` 操作创建一个很短的回调函数，但又不想用 `def` 去写一个单行函数， 而是希望通过某个快捷方式以内联方式来创建这个函数。

* 解决方案

当一些函数很简单，仅仅只是计算一个表达式的值的时候，就可以使用 `lambda` 表达式来代替了。比如：

In [46]:
add = lambda x, y: x+y
add(2, 3)

5

In [47]:
add('Hello', ' Wrold')

'Hello Wrold'

这里使用的lambda表达式跟定义函数的效果是一样的：

In [48]:
def add(x, y):
    return x + y

In [49]:
add(2, 3)

5

`lambda` 表达式典型的使用场景是排序或数据 `reduce`等等

In [50]:
names = ['David Beazley', 'Brian Jones', 'Raymond Hettinger', 'Ned Batchelder']

sorted(names, key=lambda name: name.split()[-1].lower())

['Ned Batchelder', 'David Beazley', 'Raymond Hettinger', 'Brian Jones']

* 讨论

尽管`lambda` 表达式允许你定义简单函数，但是它的使用是有限制的。

你只能指定单个表达式，它的值就是最后的返回值。也就是说不能包含其他的语言特性了，包括多个语句、条件表达式、迭代以及异常处理等等。

你可以不使用lambda表达式就能编写大部分python代码。

但是，当有人编写大量计算表达式值的短小函数或者需要用户提供回调函数的程序的时候，你就会看到lambda表达式的身影了。

### 7.7 匿名函数捕获变量值

* 问题

lambda定义了一个匿名函数，并想在定义时捕获到某些变量的值。

* 解决方案

先看下下面代码的效果：

In [51]:
x = 10
a = lambda y: x+y

In [52]:
x = 20
b = lambda y: x+ y

In [53]:
a(10)


30

In [54]:
b(10)

30

`a(10)` 和 `b(10)` 返回的结果是什么？如果你认为结果是20和30，那么你就错了。

`lambda` 表达式中的 `x` 是一个自由变量，在运行时绑定值，而不是定义时就绑定，这跟函数的默认值参数定义是不同的。

因此，在调用这个lambda表达式的时候，x的值是执行时的值。

In [55]:
x = 25
a(10)

35

In [56]:
x = 3
a(10)

13

如果你想让某个匿名函数在定义时就捕获到值，可以将那个参数值定义成默认参数即可，就像下面这样：

In [57]:
x = 10
a = lambda y, x=x: x+y

x = 20
b = lambda y, x=x: x+y

In [58]:
a(10)

20

In [59]:
b(10)

30

* 讨论

在这里列出来的问题是新手很容易犯的错误，有些新手可能会不恰当的使用lambda表达式。

比如，通过在一个循环或列表推导中创建一个lambda表达式列表，并期望函数能在定义时就记住每次的迭代值。例如：

In [60]:
funcs = [lambda x: x+n for n in range(5)]

for f in funcs:
    print(f(0))

4
4
4
4
4


In [61]:
funcs = [lambda x, n=n: x+n for n in range(5)]

for f in funcs:
    print(f(0))

0
1
2
3
4


通过使用函数**默认值参数**形式，lambda函数在定义时就能绑定到值。